In [2]:
import csv
import base64
import time
import fnmatch
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

import requests
from bs4 import BeautifulSoup

In [3]:
def crawl_dependents(repo,page_num):
    url = 'https://github.com/{}/network/dependents'.format(repo)
    dependents_data = []
    list_end = False
    
    for i in range(page_num):
        #print("GET " + url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        page_data = [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        
        for dependent in page_data:
            if dependent in dependents_data:
                list_end = True 
        
        if list_end:
            break
        else:    
            dependents_data = dependents_data + page_data
        
        try:
            paginationContainer = soup.find("div", {"class":"paginate-container"}).find_all('a')
        except:
            break
        
        try:
            if len(paginationContainer) > 1:
                paginationContainer = paginationContainer[1]
            else:
                paginationContainer = paginationContainer[0]
        except:
            break
        
        if paginationContainer:
            url = paginationContainer["href"]
        else:
            break
        
    return dependents_data

In [4]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [5]:
assert load_dotenv(), 'Environment variables could not be loaded'
g = Github(getenv("GITHUB"))

In [6]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Datasets APIs and open source projects related to Climate Change - A curated list of APIs, open data and ML/AI projects on climate change. [https://github.com/KKulma/climate-change-data] - Awesome Green Software - Research, tools, code, libraries and training to for building applications that emit less carbon into our atmosphere. [https://github.com/Green-Software-Foundation/awesome-green-software] - Awesome Sustainability Jobs - A curated list of companies in the sustainability sector that have jobs for devs. [https://github.com/pogopaule/awesome-sustainability-jobs] - Awesome Spectral Indices - A ready-to-use curated list of Spectral Indices for Remote Sensing applications. [https://github.com/davemlz/awesome-ee-spectral-indices] - Awesome Vegetation Index - List of reference, applications of common Vegetation Indices for Multi-spectral, hyper-spectral and UAV images. [https://github.com/px39n/Awesome-Vegetation-Index] - awesome-transit - Community list of transit APIs, apps, data

In [7]:
csv_projects = open("./csv/projects.csv", "w", newline="")
csv_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "platform",
    "topics",
    "rubric",
    "last_update",
    "stargazers_count",
    "number_of_dependents",
    "stars_last_year",
    "project_active_checkbox",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "created",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "open_pullrequests",
    "reviews_per_pr",
    "community_development_distribution_score",
    "last_released_date",
    "last_release_tag_name",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "organization_avatar",
    "dependents_repos"

]
writer_projects = csv.DictWriter(
    csv_projects, fieldnames=csv_fieldnames
)
writer_projects.writeheader()


csv_github_organizations_fieldnames = [
    "organization_name",
    "organization_user_name",
    "organization_github_url",
    "organization_website",
    "organization_location",
    "organization_country",
    "organization_form",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
]

csv_github_organizations = open("./csv/github_organizations.csv", "r", newline="")


github_organization_list=[]                            
reader_github_organizations = csv.DictReader(csv_github_organizations)
for entry in reader_github_organizations:
    github_organization_list.append(entry['organization_user_name'])
    
csv_github_organizations.close()
    
csv_github_organizations = open("./csv/github_organizations.csv", "a", newline="")
writer_github_organizations = csv.DictWriter(csv_github_organizations,csv_github_organizations_fieldnames)

In [8]:
for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            try:

                remaining, limit = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 300
                if remaining < min_requests:
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp()
                    countdown((int(resettime) - int(current_time) + 2))

                print(
                    "GitHub Requests | Limit: {}, Remaining: {}".format(
                        limit, remaining
                    )
                )
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                platform = "github"
                user, project_name = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                stargazers = repo.get_stargazers_with_dates()
                
                # Crawel dependents
                try:
                    dependents_repos = crawl_dependents(repo_path,20)
                    
                except Exception as e:
                    print("Dependents not available:")
                    print(e)
                    dependents_repo = []
                    
                    
                number_of_dependents = len(dependents_repos)
                dependents_repos = ",".join(dependents_repos)
                
                # Check if the project is still active
                closed_issues = repo.get_issues(state="closed")
                open_issues = repo.get_issues(state="open")

                closed_pullrequests = repo.get_pulls(state="closed")
                open_pullrequests = repo.get_pulls(state="open")
                
                closed_prs = closed_pullrequests.totalCount
                open_prs = open_pullrequests.totalCount
                
                if closed_prs > 10:
                    pr_review_analyse = 10
                else:
                    pr_review_analyse = closed_prs
                
                total_reviews = 0
                for pull_request in closed_pullrequests[0:pr_review_analyse-1]:
                    pr_reviews = pull_request.get_reviews()
                    total_reviews = total_reviews + pr_reviews.totalCount
                
                try:
                    reviews_per_pr = total_reviews/pr_review_analyse
                    
                except:                   
                    reviews_per_pr = 0

                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed

                if (
                    last_issue_closed_timedelta > timedelta(days=365)
                    and total_commits_last_year == 0
                    or repo.archived
                ):
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else:
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"

                try:
                    license = repo.get_license()
                    if license.license.spdx_id == "NOASSERTION":
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"

                labels = ",".join([entry.name for entry in repo.get_labels()])
                topics = ",".join(repo.get_topics())

                languages_states = repo.get_languages()
                programming_languages = ",".join(languages_states.keys())
                
                try:
                    dominating_language = list(languages_states.keys())[0]
                except:
                    dominating_language = ""

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:
                        if file.path == ".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print("No funding information found")
                    donation_platforms = None
                    accepts_donations = "false"

                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                except Exception as e:
                    print(e)

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(
                    sorted(contributor_activity.items(), key=lambda item: item[1])
                )
                weighted_contribution = {
                    k: v / commits_total for k, v in sorted_contributor.items()
                }

                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(
                    weighted_contribution.values()
                )

                try:
                    last_released_date = releases[0].published_at.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    )
                    last_release_tag_name = releases[0].tag_name

                except Exception as e:
                    print("No Release found")
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)

                total_number_of_commits = commits.totalCount
                
                stars_last_year = 0
                for star in stargazers:
                    starred_delta = datetime.utcnow() - star.starred_at
                    if  starred_delta < timedelta(days=365):
                        stars_last_year = stars_last_year + 1

                # Gathering organization data
                if repo.organization is None:
                    print("No Organization found.Project in user namespace.")
                    organization_user_name = None

                elif repo.organization.login not in github_organization_list:
                    print("Organization not in list. Gathering data.")
                    organization_user_name = repo.organization.login
                    organization_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    organization_location = repo.organization.location
                    organization_github_url = repo.organization.html_url
                    organization_website = repo.organization.blog
                    organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                    organization_repos = g.search_repositories(query='user:'+organization_user_name, sort='updated')     
                    organization_last_update = organization_repos[0].updated_at.strftime("%Y/%m/%d, %H:%M:%S")
                        
                        
                    organization_data = {
                        "organization_name": organization_name,
                        "organization_user_name":organization_user_name,
                        "organization_github_url":organization_github_url,
                        "organization_website":organization_website,
                        "organization_avatar": organization_avatar,
                        "organization_location": organization_location,
                        "organization_country": "",
                        "organization_form": "",
                        "organization_public_repos": organization_repos.totalCount,
                        "organization_created": organization_created,
                        "organization_last_update": organization_last_update
                    }

                    github_organization_list.append(organization_user_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)

                else:
                    organization_name = repo.organization.login
                    organization_user_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    

                project_data = {
                    "project_name": entry.name,
                    "git_namespace": user,
                    "git_url": repo.clone_url,
                    "rubric": r.key,
                    "oneliner": entry.text[2:],
                    "topics": topics,
                    "organization": organization_name,
                    "organization_user_name": organization_user_name,
                    "created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "last_update": repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "total_commits_last_year": total_commits_last_year,
                    "project_active_checkbox": project_active_checkbox,
                    "last_issue_closed": last_issue_closed.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    ),
                    "last_released_date": last_released_date,
                    "last_release_tag_name": last_release_tag_name,
                    "total_number_of_commits": total_number_of_commits,
                    "community_development_distribution_score": community_development_distribution_score,
                    "stargazers_count": repo.stargazers_count,
                    "number_of_dependents": number_of_dependents,
                    "dependents_repos": dependents_repos,
                    "stars_last_year": stars_last_year,
                    "dominating_language": dominating_language,
                    "languages": programming_languages,
                    "homepage": repo.homepage,
                    "closed_issues": closed_issues.totalCount,
                    "open_issues": open_issues.totalCount,
                    "closed_pullrequests": closed_prs,
                    "open_pullrequests": open_prs,
                    "reviews_per_pr": reviews_per_pr,
                    "good_first_issue": repo.get_issues(
                        state="open", labels=["good first issue"]
                    ).totalCount,
                    "license": license_name,
                    "contributors": repo.get_contributors().totalCount,
                    "accepts_donations": accepts_donations,
                    "donation_platforms": donation_platforms,
                    "code_of_conduct": code_of_conduct,
                    "contribution_guide": contribution_guide,
                    "organization_avatar":organization_avatar,
                    "platform":platform
                }

                print("Project Data:")
                print(project_data)
                writer_projects.writerow(project_data)

            except Exception as e:
                print(colored("Failed to gather project information:"))
                print(colored(e, "red"))
                print(traceback.format_exc())

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "gitlab"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }
            print("Project Data:")
            print(project_data)
            writer_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "custom"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "homepage": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }

            print("Project Data:")
            print(project_data)
            writer_projects.writerow(project_data)

print("------------------------")
print("All processing finished. Saving CSV files")
csv_projects.close()
csv_github_organizations.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 4996
pvlib-python is an active project
Code of conduct found
Contribution guide found
Project Data:
{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewables,photovoltaic', 'organization': 'pvlib', 'organization_user_name': 'pvlib', 'created': '2015/02/17, 00:21:33', 'last_update': '2022/02/25, 15:22:56', 'total_commits_last_year': 89, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/25, 08:26:59', 'last_released_date': '2021/09/01, 22:49:24', 'last_release_tag_name': 'v0.9.0', 'total_number_of_commits': 1380, 'community_development_distributi

{'project_name': 'pvfactors', 'git_namespace': 'SunPower', 'git_url': 'https://github.com/SunPower/pvfactors.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Open source view-factor model for diffuse shading and bifacial PV modeling.', 'topics': 'solar-energy,renewable-energy,python,bifacial', 'organization': 'SunPower', 'organization_user_name': 'SunPower', 'created': '2018/05/14, 06:10:55', 'last_update': '2022/02/19, 16:02:59', 'total_commits_last_year': 10, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/22, 21:53:38', 'last_released_date': '2022/02/22, 20:16:58', 'last_release_tag_name': 'v1.5.2', 'total_number_of_commits': 104, 'community_development_distribution_score': 0.12903225806451613, 'stargazers_count': 56, 'number_of_dependents': 5, 'dependents_repos': 'thesethtruth/LESO,prathamrg/solarapp,anomam/pvfactors_iea_pvps_study,toddkarin/vocmax,toddkarin/pvtools', 'stars_last_year': 14, 'dominating_language': 'Python', 'languages': 'Python,Makefil

{'project_name': 'Machine-Learning-for-Solar-Energy-Prediction', 'git_namespace': 'ColasGael', 'git_url': 'https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Predict the power production of a solar panel farm from weather measurements using machine learning.', 'topics': 'machine-learning,neural-network,data-processing,python,matlab,tensorflow', 'organization': 'NREL', 'organization_user_name': None, 'created': '2018/05/06, 19:43:04', 'last_update': '2022/02/23, 19:06:29', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2019/11/15, 19:01:02', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 11, 'community_development_distribution_score': 0.0, 'stargazers_count': 124, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 36, 'dominating_language': 'Python', 'languages': 'Python,MATLAB,R,TeX,TSQL', 'homepage': '', 'c

No Organization found.Project in user namespace.
Project Data:
{'project_name': 'pvtrace', 'git_namespace': 'danieljfarrell', 'git_url': 'https://github.com/danieljfarrell/pvtrace.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Optical ray tracing for luminescent materials and spectral converter photovoltaic devices.', 'topics': 'python,photovoltaics,raytracing,optics,energy', 'organization': 'pvcaptest', 'organization_user_name': None, 'created': '2011/01/05, 14:28:41', 'last_update': '2021/12/21, 09:13:10', 'total_commits_last_year': 1, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/02/18, 13:43:32', 'last_released_date': '2020/11/18, 23:04:23', 'last_release_tag_name': '2.1.6', 'total_number_of_commits': 185, 'community_development_distribution_score': 0.019108280254777066, 'stargazers_count': 79, 'number_of_dependents': 1, 'dependents_repos': 'danieljfarrell/pvtrace', 'stars_last_year': 8, 'dominating_language': 'Python', 'languages': 'Python', 'homepa

{'project_name': 'bifacial_radiance', 'git_namespace': 'NREL', 'git_url': 'https://github.com/NREL/bifacial_radiance.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Toolkit for working with RADIANCE for the ray-trace modeling of Bifacial Photovoltaics.', 'topics': 'radiance,bifacial,photovoltaics,renewable-energy,renewables,gui', 'organization': 'NREL', 'organization_user_name': 'National Renewable Energy Laboratory', 'created': '2017/12/13, 17:56:18', 'last_update': '2022/02/23, 20:12:08', 'total_commits_last_year': 319, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/21, 13:41:11', 'last_released_date': '2022/02/14, 21:04:38', 'last_release_tag_name': 'v0.4.0-dev2', 'total_number_of_commits': 1429, 'community_development_distribution_score': 0.36534839924670437, 'stargazers_count': 54, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 15, 'dominating_language': 'Python', 'languages': 'Python,C++,TeX,C,Shell,Makefile,Awk', 'homepage'

...

{'project_name': 'Awesome Sustainability Jobs', 'git_namespace': 'pogopaule', 'git_url': 'https://github.com/pogopaule/awesome-sustainability-jobs.git', 'rubric': 'Curated Lists', 'oneliner': 'A curated list of companies in the sustainability sector that have jobs for devs.', 'topics': 'awesome-list,sustainability,jobs,jobsearch,job-search,renewable-energies,renewables,circular-economy,emobility,job-portal,jobseeker,job-board,carbon-footprint,climate-change,climatechange,green-tech,climate-action', 'organization': 'Green-Software-Foundation', 'organization_user_name': None, 'created': '2020/11/22, 21:00:23', 'last_update': '2022/02/21, 01:06:49', 'total_commits_last_year': 327, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/12/06, 09:32:40', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 363, 'community_development_distribution_score': 0.030726256983240274, 'stargazers_count': 146, 'number_of_dependents': 0, 'dependents_repos': '', 'sta

{'project_name': 'Awesome Coastal', 'git_namespace': 'chrisleaman', 'git_url': 'https://github.com/chrisleaman/awesome-coastal.git', 'rubric': 'Curated Lists', 'oneliner': 'A curated list of awesome resources for coastal engineers and scientists.', 'topics': 'awesome,coastal,beaches,coastal-modelling,coastal-engineering', 'organization': 'CUTR-at-USF', 'organization_user_name': None, 'created': '2019/05/07, 23:47:21', 'last_update': '2021/12/09, 12:20:50', 'total_commits_last_year': 0, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/08/31, 05:36:33', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 32, 'community_development_distribution_score': 0.11111111111111116, 'stargazers_count': 49, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 13, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 24, 'open_issues': 6, 'closed_pullrequests': 3, 'open_pullrequests': 0, 'reviews_per_pr': 0.0, 'good_fi

{'project_name': 'Awesome Clean Tech', 'git_namespace': 'nglgzz', 'git_url': 'https://github.com/nglgzz/awesome-clean-tech.git', 'rubric': 'Curated Lists', 'oneliner': 'Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities.', 'topics': 'awesome,awesome-list,clean-tech,environment-friendly,climate-change,climate-change-mitigation', 'organization': 'awesome-cryosphere', 'organization_user_name': None, 'created': '2019/07/31, 19:54:41', 'last_update': '2022/02/26, 12:38:30', 'total_commits_last_year': 1, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/09/06, 07:12:09', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 60, 'community_development_distribution_score': 0.5593220338983051, 'stargazers_count': 332, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 

{'project_name': 'Code against Climate Change', 'git_namespace': 'daviddao', 'git_url': 'https://github.com/daviddao/code-against-climate-change.git', 'rubric': 'Curated Lists', 'oneliner': 'A curated list of tech projects against climate change - hoping to inspire disruptive technological climate action.', 'topics': '', 'organization': 'softwareunderground', 'organization_user_name': None, 'created': '2018/12/29, 23:37:54', 'last_update': '2022/02/18, 06:55:44', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2020/10/13, 13:12:58', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 9, 'community_development_distribution_score': 0.5, 'stargazers_count': 160, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 46, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 3, 'open_issues': 0, 'closed_pullrequests': 3, 'open_pullrequests': 0, 'reviews_per_pr': 0.0, 'good_first_issue

awesome-gee-community-datasets is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 2034
awesome-gee-community-datasets is an active project
No Release found
list index out of range
No Organization found.Project in user namespace.
Project Data:
{'project_name': 'awesome-gee-community-datasets', 'git_namespace': 'samapriya', 'git_url': 'https://github.com/samapriya/awesome-gee-community-datasets.git', 'rubric': 'Curated Lists', 'oneliner': 'Community Datasets & Data Commons in Google Earth Engine.', 'topics': 'googleearthengine,community-datasets,catalog', 'organization': 'arctic-risk', 'organization_user_name': None, 'created': '2020/05/08, 08:33:41', 'last_update': '2022/02/25, 21:06:51', 'total_commits_last_year': 296, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/23, 18:51:06', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 331, 'community_development_distribution_score': 0.2930513595166163, 'stargazers_count': 194, 'num